In [1]:
import pandas as pd
import requests
import json
import os
from bs4 import BeautifulSoup
import re
import numpy as np
from ast import literal_eval
from queue import Queue
from threading import Thread
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
json1_file = open(os.getcwd() + '/credentials.json')
json1_str = json1_file.read()
credential = json.loads(json1_str)
Genius_Token = credential['Genius_API']['Access_Token']

In [3]:
tracks_with_song_ids = pd.read_csv(r'song_id_no_duplicates.csv')

/Users/Stella/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
tracks_with_song_ids['song_id'].duplicated().any()

False

In [3]:
def request_song_info(song_id,Genius_Token):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + Genius_Token}
    song_url = base_url + '/songs/' + str(song_id)
    response = requests.get(song_url, headers = headers,verify=False)

    return response

In [5]:
def save_song_info(song_id):
    response = request_song_info(song_id,Genius_Token)
    if response.ok == True:
        song_info = response.json()
        return song_info
    else:
        return "Not Found"
        

In [6]:
def worker():
    while True:
        item = q.get()
        if save_song_info(item[0]) is not None:
            result.append([item[0],save_song_info(item[0])])
            q.task_done()

In [ ]:
result = list()
q = Queue()

for i in range(8):
    t = Thread(target=worker)
    t.daemon = True
    t.start()

for item in tracks_with_song_ids.iterrows():
    q.put([item[1]['song_id']])
    
q.join() 



In [ ]:
df = pd.DataFrame(columns = {"song_id":[i[0] for i in result],"song_info":[i[1] for i in result]})
df_dedupe = df[df['song_id'].duplicated() == False]
df_dedupe.to_csv(r'Complete Song Info(Genius).csv',index = False)

In [26]:
def update_keys(line):
    result = dict()
    keep_attributes = ['recording_location','producer_artists','release_date','writer_artists']
    for key in keep_attributes:
        if key in line.keys():
            result[key] = line[key]
            
    return result

In [39]:
df['song_info'] = df['song_info'].apply(lambda x: literal_eval(x) if len(x) != 0 else "Not Found")

In [40]:
df['update_info'] = df['song_info'].apply(lambda x: update_keys(x))

In [78]:
df['song_id'].duplicated().any()

False

In [ ]:
df.to_csv('update_info.csv',index = False)